In [ ]:
#| default_exp fast_gsc

In [ ]:
# | export
# Google Search Console API Integration
# > FastAPI endpoints for OAuth authentication and date retrieval from Google Search
import pickle
from datetime import datetime, timedelta
from pathlib import Path
from enum import Enum
from typing import List, Optional, Tuple, Any, Dict, Union

# FastAPI imports
from fastapi import FastAPI, HTTPException, Query
from fastapi.requests import Request as FastAPIRequest
from fastapi.responses import RedirectResponse

# Google Auth imports
from google.oauth2.credentials import Credentials
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request as GoogleRequest

# Pydantic imports
from pydantic import BaseModel, Field, HttpUrl

from bson import ObjectId

# Local imports
from seo_rat.gsc_db import SearchConsoleDB

In [ ]:
#| export
app = FastAPI()
db = SearchConsoleDB()

In [ ]:
#| export
class AuthResponse(BaseModel):
    message: str


class RedirectURL(BaseModel):
    url: str


In [ ]:
#| export
class GSCAuth:
    """Google Search Console Authentication Handler with persistent storage."""

    def __init__(
        self,
        secrets_file: str = "./client_secrets.json",
        token_file: str = "./token.pickle",  # File to store credentials
        callback_url: str = "http://localhost:8000/callback",
    ):
        self.secrets_file = secrets_file
        self.token_file = Path(token_file)
        self.callback_url = callback_url
        self.credentials = self._load_credentials()

    def _load_credentials(self) -> Optional[Credentials]:
        """Load credentials from file if they exist."""
        if self.token_file.exists():
            with open(self.token_file, "rb") as token:
                return pickle.load(token)
        return None

    def _save_credentials(self, credentials: Credentials):
        """Save credentials to file"""
        with open(self.token_file, "wb") as token:
            pickle.dump(credentials, token)

    def get_credentials(self) -> Optional[Credentials]:
        """Get current credentials, refreshing if necessary"""
        if not self.credentials:
            return None

        if self.credentials.expired:
            self.credentials.refresh(GoogleRequest())
            self._save_credentials(self.credentials)

        return self.credentials

    def set_credentials(self, credentials: Credentials):
        """Set and save new credentials"""
        self.credentials = credentials
        self._save_credentials(credentials)

    def initialize_flow(self) -> InstalledAppFlow:
        """Initialize OAuth2 flow for Google Search Console"""
        return InstalledAppFlow.from_client_secrets_file(
            self.secrets_file,
            scopes=[
                "https://www.googleapis.com/auth/webmasters.readonly",
                "https://www.googleapis.com/auth/webmasters",
            ],
            redirect_uri=self.callback_url,
        )


# Initialize auth
auth = GSCAuth()


# Dependency for routes that need authentication
async def require_auth():
    """Dependency to check for valid authentication"""
    credentials = auth.get_credentials()
    if not credentials:
        raise HTTPException(
            status_code=401, detail="Not authenticated. Please visit /login first."
        )
    return credentials

In [ ]:
#| export
@app.get("/login", tags=["gsc_api"], response_model=Union[AuthResponse, RedirectURL])
async def login():
    """Start OAuth2 flow if not already authenticated."""
    if auth.get_credentials():
        return AuthResponse(message="Already authenticated")

    flow = auth.initialize_flow()
    authorization_url, _ = flow.authorization_url(
        access_type="offline",
        include_granted_scopes="true",
        prompt="consent",
    )
    return RedirectURL(url=authorization_url)

In [ ]:
#| export
@app.get("/callback", tags=["gsc_api"], response_model=RedirectURL)
async def callback(request: FastAPIRequest):
    """Handle OAuth callback and store credentials"""
    flow = auth.initialize_flow()
    flow.fetch_token(authorization_response=str(request.url))
    auth.set_credentials(flow.credentials)
    return RedirectURL(url="/sites")

In [ ]:
#| export
class SiteInfo(BaseModel):
    """Site information from Google Search Console.

    Attributes:
        site_url: URL of the site
        permission_level: User's permission level for the site

    """

    site_url: str
    permission_level: str


@app.get(
    "/sites",
    response_model=List[SiteInfo],
    summary="Get verified sites",
    description="Retrieve list of verified sites from Google Search Console",
    tags=["gsc_api"]
)
async def get_sites() -> List[SiteInfo]:
    """Get list of verified sites from Google Search Console.

    This endpoint retrieves all sites that the authenticated user has access to,
    filtering out unverified sites. Each site includes its URL and the user's
    permission level.

    Returns:
        List[SiteInfo]: List of verified sites and their permission levels

    Raises:
        HTTPException:
            - 401 if not authenticated
            - 500 if Google API request fails

    Examples:
        >>> response = await get_sites()
        >>> response[0]
        {
            "site_url": "sc-domain:example.com",
            "permission_level": "siteOwner"
        }

    Notes:
        Requires authentication via /login endpoint first

    """
    # Check authentication
    if not auth.credentials:
        raise HTTPException(
            status_code=401,
            detail="Not authenticated. Please visit /login first.",
        )

    try:
        # Initialize service
        service = build("searchconsole", "v1", credentials=auth.credentials)

        # Get site list
        sites_list = service.sites().list().execute()

        # Filter and format verified sites
        verified_sites = [
            SiteInfo(site_url=site["siteUrl"], permission_level=site["permissionLevel"])
            for site in sites_list.get("siteEntry", [])
            if site["permissionLevel"] != "siteUnverifiedUser"
        ]

        return verified_sites

    except Exception as e:
        raise HTTPException(
            status_code=500,
            detail=f"Failed to retrieve sites: {e!s}",
        )

In [ ]:
#| export
def serialize_mongodb_doc(
    doc: Union[Dict, List, ObjectId, datetime, Any],
) -> Union[Dict, List, str, Any]:
    """Serialize MongoDB documents for JSON compatibility.

    Recursively converts MongoDB-specific types to JSON-serializable formats:
    - ObjectId to string
    - datetime to ISO format string
    - Nested dictionaries and lists are processed recursively

    Args:
        doc: MongoDB document or field to serialize
            Can be dict, list, ObjectId, datetime, or any other type

    Returns:
        Serialized version of the input, safe for JSON conversion

    Examples:
        >>> from bson import ObjectId
        >>> from datetime import datetime
        >>> doc = {
        ...     "_id": ObjectId("507f1f77bcf86cd799439011"),
        ...     "timestamp": datetime(2024, 1, 1),
        ...     "nested": {"date": datetime(2024, 1, 2)}
        ... }
        >>> serialized = serialize_mongodb_doc(doc)
        >>> serialized
        {
            '_id': '507f1f77bcf86cd799439011',
            'timestamp': '2024-01-01T00:00:00',
            'nested': {'date': '2024-01-02T00:00:00'}
        }

    Notes:
        - ObjectId is converted to string using str()
        - datetime is converted to ISO format string
        - Other types are returned as-is
        - Handles nested structures recursively
    """
    # Handle dictionaries recursively
    if isinstance(doc, dict):
        return {k: serialize_mongodb_doc(v) for k, v in doc.items()}

    # Handle lists recursively
    if isinstance(doc, list):
        return [serialize_mongodb_doc(item) for item in doc]

    # Convert ObjectId to string
    if isinstance(doc, ObjectId):
        return str(doc)

    # Convert datetime to ISO format
    if isinstance(doc, datetime):
        return doc.isoformat()

    # Return other types as-is
    return doc


In [ ]:
#| export


def get_date_range(
    range_type: str = "today",  # Type of date range to generate
    days: Optional[int] = None,  # Number of days for last_days range
    months: Optional[int] = None,  # Number of months for last_months range
    start_date: Optional[str] = None,  # Custom start date in YYYY-MM-DD format
    end_date: Optional[str] = None,  # Custom end date in YYYY-MM-DD format
) -> Tuple[str, str]:
    """Generate date range for Google Search Console analytics queries.

    Handles different types of date ranges while accounting for Google Search Console's
    3-day data delay. All dates are returned in YYYY-MM-DD format.

    Args:
        range_type: Type of date range to generate
            - 'today': Returns date from 3 days ago
            - 'last_days': Returns range for specified number of days
            - 'last_months': Returns range for specified number of months
            - 'custom': Returns range between specified dates
        days: Required for 'last_days', number of days to look back
        months: Required for 'last_months', number of months to look back
        start_date: Required for 'custom', start date in YYYY-MM-DD format
        end_date: Required for 'custom', end date in YYYY-MM-DD format

    Returns:
        Tuple containing start and end dates in YYYY-MM-DD format

    Raises:
        HTTPException:
            - 400 if parameters are invalid
            - 400 if start_date is after end_date

    Examples:
        >>> get_date_range("today")
        ('2024-01-07', '2024-01-07')  # If today is 2024-01-10

        >>> get_date_range("last_days", days=7)
        ('2024-01-01', '2024-01-07')

        >>> get_date_range("last_months", months=1)
        ('2023-12-07', '2024-01-07')

        >>> get_date_range("custom",
        ...               start_date="2024-01-01",
        ...               end_date="2024-01-07")
        ('2024-01-01', '2024-01-07')

    Notes:
        - Google Search Console data has a 3-day delay
        - Months are approximated as 30 days
        - Custom date ranges are adjusted if they include unavailable dates
    """
    try:
        # Calculate available date range
        today = datetime.now()
        latest_available = today - timedelta(days=3)  # Account for GSC delay

        if range_type == "today":
            # Single day (3 days ago)
            target_date = latest_available.strftime("%Y-%m-%d")
            return target_date, target_date

        if range_type == "last_days" and days:
            # Last N days
            end_date = latest_available
            start_date = end_date - timedelta(days=days)
            return start_date.strftime("%Y-%m-%d"), end_date.strftime("%Y-%m-%d")

        if range_type == "last_months" and months:
            # Last N months (30 days per month)
            end_date = latest_available
            start_date = end_date - timedelta(days=30 * months)
            return start_date.strftime("%Y-%m-%d"), end_date.strftime("%Y-%m-%d")

        if range_type == "custom" and start_date and end_date:
            # Custom date range
            start = datetime.strptime(start_date, "%Y-%m-%d")
            end = datetime.strptime(end_date, "%Y-%m-%d")

            # Validate dates
            if end > latest_available.date():
                end = latest_available
            if start > end:
                raise ValueError("Start date must be before end date")

            return start.strftime("%Y-%m-%d"), end.strftime("%Y-%m-%d")

        raise ValueError("Invalid date range parameters")

    except Exception as e:
        raise HTTPException(status_code=400, detail=f"Date range error: {str(e)}")


In [ ]:
#| export
class AnalyticsResponse(BaseModel):
    """Response model for analytics data

    Attributes:
        site_url: Website being analyzed
        date_range: Information about the analysis period
        grouped_by: Dimension used for grouping
        sorted_by: Metric used for sorting
        total_results: Number of results returned
        results: Detailed analytics data
        summary: Aggregated metrics
    """

    site_url: str
    date_range: Dict[str, Any]
    grouped_by: str
    sorted_by: str
    total_results: int
    results: List[Dict]
    summary: Dict[str, float]


@app.get(
    "/analyze-analytics/{site_url}",
    response_model=AnalyticsResponse,
    summary="Analyze site analytics data",
    description="Analyze Google Search Console data with flexible grouping and date ranges",
    tags=["analytics"]
)
async def analyze_site_analytics(
    site_url: str,  # Website URL to analyze
    range_type: str = Query(
        default="today",
        description="Date range type: today, last_days, last_months, custom",
    ),
    days: Optional[int] = Query(
        default=None, description="Number of days for last_days range", ge=1, le=365
    ),
    months: Optional[int] = Query(
        default=None, description="Number of months for last_months range", ge=1, le=12
    ),
    start_date: Optional[str] = Query(
        default=None, description="Custom start date (YYYY-MM-DD)"
    ),
    end_date: Optional[str] = Query(
        default=None, description="Custom end date (YYYY-MM-DD)"
    ),
    group_by: str = Query(
        default="query",
        description="Dimension to group by: query, page, country, device",
    ),
    sort_by: str = Query(
        default="clicks",
        description="Metric to sort by: clicks, impressions, position, ctr",
    ),
    limit: int = Query(
        default=10, le=100, description="Maximum number of results to return"
    ),
) -> AnalyticsResponse:
    """Analyze Google Search Console data with flexible grouping and date ranges.

    Retrieves and analyzes stored analytics data, supporting various grouping
    dimensions and date ranges. Calculates aggregate metrics and provides
    detailed breakdowns.

    Args:
        site_url: Website to analyze
        range_type: Type of date range (today, last_days, last_months, custom)
        days: Number of days for last_days range
        months: Number of months for last_months range
        start_date: Start date for custom range (YYYY-MM-DD)
        end_date: End date for custom range (YYYY-MM-DD)
        group_by: Dimension to group results by
        sort_by: Metric to sort results by
        limit: Maximum number of results to return

    Returns:
        AnalyticsResponse containing:
        - Date range information
        - Grouped and sorted results
        - Summary metrics

    Raises:
        HTTPException:
            - 400 for invalid date ranges
            - 500 for database errors

    Examples:
        >>> # Get top queries from last 7 days
        >>> response = await analyze_site_analytics(
        ...     "sc-domain:example.com",
        ...     range_type="last_days",
        ...     days=7,
        ...     group_by="query"
        ... )

        >>> # Get country performance for a custom date range
        >>> response = await analyze_site_analytics(
        ...     "sc-domain:example.com",
        ...     range_type="custom",
        ...     start_date="2024-01-01",
        ...     end_date="2024-01-31",
        ...     group_by="country"
        ... )

    Notes:
        - Dates are adjusted for Google Search Console's 3-day delay
        - Position metrics are weighted by impressions
        - CTR is calculated as clicks/impressions
    """
    try:

        # Match site and date range
        match_stage = {
            "site_url": site_url,
            "date": {
                "$gte": start_date,
                "$lte": end_date,
            },
        }

        pipeline = [
            {"$match": match_stage},
            {
                "$group": {
                    "_id": f"$dimensions.{group_by}",
                    "clicks": {"$sum": "$metrics.clicks"},
                    "impressions": {"$sum": "$metrics.impressions"},
                    "avg_position": {
                        "$avg": {
                            "$multiply": ["$metrics.position", "$metrics.impressions"],
                        },
                    },
                    "total_impressions": {"$sum": "$metrics.impressions"},
                    "dates": {"$addToSet": "$date"},
                    "pages": {"$addToSet": "$dimensions.page"},
                    "countries": {"$addToSet": "$dimensions.country"},
                    "devices": {"$addToSet": "$dimensions.device"},
                },
            },
            {
                "$addFields": {
                    "avg_position": {
                        "$cond": [
                            {"$eq": ["$total_impressions", 0]},
                            0,
                            {"$divide": ["$avg_position", "$total_impressions"]},
                        ],
                    },
                    "ctr": {
                        "$cond": [
                            {"$eq": ["$total_impressions", 0]},
                            0,
                            {"$divide": ["$clicks", "$total_impressions"]},
                        ],
                    },
                    "date_count": {"$size": "$dates"},
                },
            },
            {"$sort": {sort_by: -1}},
            {"$limit": limit},
        ]

        results = list(db.db.analytics.aggregate(pipeline))

        # Calculate overall metrics
        total_clicks = sum(r["clicks"] for r in results)
        total_impressions = sum(r["impressions"] for r in results)

        return {
            "site_url": site_url,
            "date_range": {
                "start": start_date,
                "end": end_date,
                "type": range_type,
                "total_days": len(set(date for r in results for date in r["dates"])),
            },
            "grouped_by": group_by,
            "sorted_by": sort_by,
            "total_results": len(results),
            "results": results,
            "summary": {
                "total_clicks": total_clicks,
                "total_impressions": total_impressions,
                "average_position": (
                    sum(r["avg_position"] * r["impressions"] for r in results)
                    / total_impressions
                    if total_impressions > 0
                    else 0
                ),
                "average_ctr": (
                    total_clicks / total_impressions if total_impressions > 0 else 0
                ),
            },
        }

    except Exception as e:
        raise HTTPException(status_code=500, detail=str(e))


In [ ]:
#| export


class MetricsData(BaseModel):
    """Base metrics data model

    Attributes:
        clicks: Number of clicks
        impressions: Number of impressions
        avg_position: Average position weighted by impressions
        ctr: Click-through rate (clicks/impressions)
    """

    clicks: int
    impressions: int
    avg_position: float
    ctr: float


class QueryPerformance(MetricsData):
    """Query performance data with dimensions

    Attributes:
        query: Search query text
        dates: List of dates with data
        pages: List of pages ranking for query
        countries: List of countries with data
        devices: List of devices with data
    """

    query: str = Field(..., description="Search query")
    dates: List[str] = Field(default_factory=list)
    pages: List[str] = Field(default_factory=list)
    countries: List[str] = Field(default_factory=list)
    devices: List[str] = Field(default_factory=list)


class QueryAnalyticsResponse(BaseModel):
    """Response model for query analytics

    Attributes:
        site_url: Website being analyzed
        query: Search query being analyzed
        date_range: Information about the analysis period
        main_metrics: Overall query performance
        pages_performance: Optional breakdown by page
        country_performance: Optional breakdown by country
    """

    site_url: str
    query: str
    date_range: Dict[str, Any]
    main_metrics: Optional[QueryPerformance]
    pages_performance: Optional[List[MetricsData]]
    country_performance: Optional[List[MetricsData]]


@app.get(
    "/search-query-analytics/{site_url}",
    response_model=QueryAnalyticsResponse,
    summary="Analyze query performance",
    description="Analyze performance of specific search query with breakdowns",
    tags=["analytics"]
)
async def search_query_analytics(
    site_url: str,  # Website URL to analyze
    query: str = Query(
        ...,  # Search query to analyze
        min_length=1,
        description="Search query to analyze",
    ),
    range_type: str = Query(
        default="today",
        description="Date range type: today, last_days, last_months, custom",
    ),
    days: Optional[int] = Query(
        default=None, description="Number of days for last_days range", ge=1, le=365
    ),
    months: Optional[int] = Query(
        default=None, description="Number of months for last_months range", ge=1, le=12
    ),
    start_date: Optional[str] = Query(
        default=None, description="Custom start date (YYYY-MM-DD)"
    ),
    end_date: Optional[str] = Query(
        default=None, description="Custom end date (YYYY-MM-DD)"
    ),
    show_pages: bool = Query(default=True, description="Include page-level breakdown"),
    show_countries: bool = Query(
        default=False, description="Include country-level breakdown"
    ),
) -> QueryAnalyticsResponse:
    """Analyze performance of specific search query over time.

    Retrieves detailed performance metrics for a search query, including
    optional breakdowns by page and country. All metrics are weighted
    by impressions where appropriate.

    Args:
        site_url: Website to analyze
        query: Search query to analyze
        range_type: Type of date range (today, last_days, last_months, custom)
        days: Number of days for last_days range
        months: Number of months for last_months range
        start_date: Start date for custom range (YYYY-MM-DD)
        end_date: End date for custom range (YYYY-MM-DD)
        show_pages: Include page-level breakdown
        show_countries: Include country-level breakdown

    Returns:
        QueryAnalyticsResponse containing:
        - Overall query performance
        - Optional page breakdown
        - Optional country breakdown

    Raises:
        HTTPException:
            - 400 for invalid parameters
            - 500 for database errors

    Examples:
        >>> # Analyze query for last 7 days with page breakdown
        >>> response = await search_query_analytics(
        ...     "sc-domain:example.com",
        ...     query="example search",
        ...     range_type="last_days",
        ...     days=7,
        ...     show_pages=True
        ... )

        >>> # Analyze query with country breakdown
        >>> response = await search_query_analytics(
        ...     "sc-domain:example.com",
        ...     query="example search",
        ...     show_countries=True
        ... )

    Notes:
        - Query matching is case-insensitive
        - Position metrics are weighted by impressions
        - CTR is calculated as clicks/impressions
    """
    try:
        # Get date range using utility function
        start_date, end_date = get_date_range(
            range_type=range_type,
            days=days,
            months=months,
            start_date=start_date,
            end_date=end_date,
        )

        # Base match condition
        match_stage = {
            "site_url": site_url,
            "date": {
                "$gte": start_date,
                "$lte": end_date,
            },
            "dimensions.query": {"$regex": query, "$options": "i"},
        }

        pipeline = [
            {"$match": match_stage},
            {
                "$group": {
                    "_id": "$dimensions.query",
                    "clicks": {"$sum": "$metrics.clicks"},
                    "impressions": {"$sum": "$metrics.impressions"},
                    "avg_position": {
                        "$avg": {
                            "$multiply": ["$metrics.position", "$metrics.impressions"],
                        },
                    },
                    "total_impressions": {"$sum": "$metrics.impressions"},
                    "dates": {"$addToSet": "$date"},
                    "pages": {"$addToSet": "$dimensions.page"},
                    "countries": {"$addToSet": "$dimensions.country"},
                    "devices": {"$addToSet": "$dimensions.device"},
                },
            },
            {
                "$addFields": {
                    "avg_position": {
                        "$cond": [
                            {"$eq": ["$total_impressions", 0]},
                            0,
                            {"$divide": ["$avg_position", "$total_impressions"]},
                        ],
                    },
                    "ctr": {
                        "$cond": [
                            {"$eq": ["$total_impressions", 0]},
                            0,
                            {"$divide": ["$clicks", "$total_impressions"]},
                        ],
                    },
                    "date_count": {"$size": "$dates"},
                },
            },
        ]

        results = list(db.db.analytics.aggregate(pipeline))

        response = {
            "site_url": site_url,
            "query": query,
            "date_range": {
                "start": start_date,
                "end": end_date,
                "type": range_type,
                "total_days": len(set(date for r in results for date in r["dates"])),
            },
            "main_metrics": results[0] if results else None,
        }

        if show_pages and results:
            # Get page-specific performance for this query
            page_pipeline = [
                {"$match": match_stage},
                {
                    "$group": {
                        "_id": "$dimensions.page",
                        "clicks": {"$sum": "$metrics.clicks"},
                        "impressions": {"$sum": "$metrics.impressions"},
                        "avg_position": {
                            "$avg": {
                                "$multiply": [
                                    "$metrics.position",
                                    "$metrics.impressions",
                                ],
                            },
                        },
                        "total_impressions": {"$sum": "$metrics.impressions"},
                        "dates": {"$addToSet": "$date"},
                    },
                },
                {
                    "$addFields": {
                        "avg_position": {
                            "$cond": [
                                {"$eq": ["$total_impressions", 0]},
                                0,
                                {"$divide": ["$avg_position", "$total_impressions"]},
                            ],
                        },
                        "ctr": {
                            "$cond": [
                                {"$eq": ["$total_impressions", 0]},
                                0,
                                {"$divide": ["$clicks", "$total_impressions"]},
                            ],
                        },
                    },
                },
                {"$sort": {"clicks": -1}},
            ]
            response["pages_performance"] = list(
                db.db.analytics.aggregate(page_pipeline)
            )

        if show_countries and results:
            # Get country breakdown
            country_pipeline = [
                {"$match": match_stage},
                {
                    "$group": {
                        "_id": "$dimensions.country",
                        "clicks": {"$sum": "$metrics.clicks"},
                        "impressions": {"$sum": "$metrics.impressions"},
                        "avg_position": {
                            "$avg": {
                                "$multiply": [
                                    "$metrics.position",
                                    "$metrics.impressions",
                                ],
                            },
                        },
                        "total_impressions": {"$sum": "$metrics.impressions"},
                        "dates": {"$addToSet": "$date"},
                    },
                },
                {
                    "$addFields": {
                        "avg_position": {
                            "$cond": [
                                {"$eq": ["$total_impressions", 0]},
                                0,
                                {"$divide": ["$avg_position", "$total_impressions"]},
                            ],
                        },
                        "ctr": {
                            "$cond": [
                                {"$eq": ["$total_impressions", 0]},
                                0,
                                {"$divide": ["$clicks", "$total_impressions"]},
                            ],
                        },
                    },
                },
                {"$sort": {"clicks": -1}},
            ]
            response["country_performance"] = list(
                db.db.analytics.aggregate(country_pipeline)
            )

        return response

    except Exception as e:
        raise HTTPException(status_code=500, detail=str(e))


In [ ]:
#| export


class DeviceType(str, Enum):
    """Device types from Google Search Console"""

    MOBILE = "MOBILE"
    DESKTOP = "DESKTOP"
    TABLET = "TABLET"


class MetricsData(BaseModel):
    """Base metrics data model"""

    clicks: int = Field(ge=0, description="Number of clicks")
    impressions: int = Field(ge=0, description="Number of impressions")
    avg_position: float = Field(
        ge=0, description="Average position weighted by impressions"
    )
    ctr: float = Field(ge=0, le=1, description="Click-through rate")


class PagePerformance(MetricsData):
    """Page performance data with dimensions"""

    page: str = Field(..., description="Page URL path")
    dates: List[str] = Field(default_factory=list, description="Dates with data")
    queries: List[str] = Field(
        default_factory=list, description="Queries leading to page"
    )
    countries: List[str] = Field(
        default_factory=list, description="Countries with traffic"
    )
    devices: List[DeviceType] = Field(default_factory=list, description="Devices used")


class PageAnalyticsResponse(BaseModel):
    """Response model for page analytics

    Attributes:
        site_url: Website being analyzed
        page_url: Page URL being analyzed
        date_range: Information about the analysis period
        main_metrics: Overall page performance
        query_performance: Optional breakdown by query
        country_performance: Optional breakdown by country
        device_performance: Optional breakdown by device
    """

    site_url: str
    page_url: str
    date_range: Dict[str, Any]
    main_metrics: Optional[PagePerformance]
    query_performance: Optional[List[MetricsData]]
    country_performance: Optional[List[MetricsData]]
    device_performance: Optional[List[MetricsData]]


@app.get(
    "/search-page-analytics/{site_url}",
    response_model=PageAnalyticsResponse,
    summary="Analyze page performance",
    description="Analyze performance of specific page with detailed breakdowns",
    tags=["analytics"]
)
async def search_page_analytics(
    site_url: str,  # Website URL to analyze
    page_url: str = Query(
        ...,  # Page URL to analyze
        min_length=1,
        description="Page URL or path to analyze",
    ),
    range_type: str = Query(
        default="today",
        description="Date range type: today, last_days, last_months, custom",
    ),
    days: Optional[int] = Query(
        default=None, ge=1, le=365, description="Number of days for last_days range"
    ),
    months: Optional[int] = Query(
        default=None, ge=1, le=12, description="Number of months for last_months range"
    ),
    start_date: Optional[str] = Query(
        default=None, description="Custom start date (YYYY-MM-DD)"
    ),
    end_date: Optional[str] = Query(
        default=None, description="Custom end date (YYYY-MM-DD)"
    ),
    show_queries: bool = Query(
        default=True, description="Include query-level breakdown"
    ),
    show_countries: bool = Query(
        default=False, description="Include country-level breakdown"
    ),
    show_devices: bool = Query(
        default=False, description="Include device-level breakdown"
    ),
) -> PageAnalyticsResponse:
    """Analyze performance of specific page over time.

    Retrieves detailed performance metrics for a page URL, including
    optional breakdowns by query, country, and device. All metrics are
    weighted by impressions where appropriate.

    Args:
        site_url: Website to analyze
        page_url: Page URL or path to analyze
        range_type: Type of date range (today, last_days, last_months, custom)
        days: Number of days for last_days range
        months: Number of months for last_months range
        start_date: Start date for custom range (YYYY-MM-DD)
        end_date: End date for custom range (YYYY-MM-DD)
        show_queries: Include query-level breakdown
        show_countries: Include country-level breakdown
        show_devices: Include device-level breakdown

    Returns:
        PageAnalyticsResponse containing:
        - Overall page performance
        - Optional query breakdown
        - Optional country breakdown
        - Optional device breakdown

    Raises:
        HTTPException:
            - 400 for invalid parameters
            - 500 for database errors

    Examples:
        >>> # Analyze page for last 7 days with query breakdown
        >>> response = await search_page_analytics(
        ...     "sc-domain:example.com",
        ...     page_url="/example-page",
        ...     range_type="last_days",
        ...     days=7,
        ...     show_queries=True
        ... )

        >>> # Full breakdown analysis
        >>> response = await search_page_analytics(
        ...     "sc-domain:example.com",
        ...     page_url="/example-page",
        ...     show_queries=True,
        ...     show_countries=True,
        ...     show_devices=True
        ... )

    Notes:
        - URL matching is case-insensitive
        - Position metrics are weighted by impressions
        - CTR is calculated as clicks/impressions
        - Device types are normalized to MOBILE, DESKTOP, TABLET
    """
    try:
        # Get date range using utility function
        start_date, end_date = get_date_range(
            range_type=range_type,
            days=days,
            months=months,
            start_date=start_date,
            end_date=end_date,
        )

        # Base match condition
        match_stage = {
            "site_url": site_url,
            "date": {
                "$gte": start_date,
                "$lte": end_date,
            },
            "dimensions.page": {"$regex": page_url, "$options": "i"},
        }

        pipeline = [
            {"$match": match_stage},
            {
                "$group": {
                    "_id": "$dimensions.page",
                    "clicks": {"$sum": "$metrics.clicks"},
                    "impressions": {"$sum": "$metrics.impressions"},
                    "avg_position": {
                        "$avg": {
                            "$multiply": ["$metrics.position", "$metrics.impressions"],
                        },
                    },
                    "total_impressions": {"$sum": "$metrics.impressions"},
                    "dates": {"$addToSet": "$date"},
                    "queries": {"$addToSet": "$dimensions.query"},
                    "countries": {"$addToSet": "$dimensions.country"},
                    "devices": {"$addToSet": "$dimensions.device"},
                },
            },
            {
                "$addFields": {
                    "avg_position": {
                        "$cond": [
                            {"$eq": ["$total_impressions", 0]},
                            0,
                            {"$divide": ["$avg_position", "$total_impressions"]},
                        ],
                    },
                    "ctr": {
                        "$cond": [
                            {"$eq": ["$total_impressions", 0]},
                            0,
                            {"$divide": ["$clicks", "$total_impressions"]},
                        ],
                    },
                    "date_count": {"$size": "$dates"},
                },
            },
        ]

        results = list(db.db.analytics.aggregate(pipeline))

        response = {
            "site_url": site_url,
            "page_url": page_url,
            "date_range": {
                "start": start_date,
                "end": end_date,
                "type": range_type,
                "total_days": len(set(date for r in results for date in r["dates"])),
            },
            "main_metrics": results[0] if results else None,
        }

        if show_queries and results:
            # Get query performance for this page
            query_pipeline = [
                {"$match": match_stage},
                {
                    "$group": {
                        "_id": "$dimensions.query",
                        "clicks": {"$sum": "$metrics.clicks"},
                        "impressions": {"$sum": "$metrics.impressions"},
                        "avg_position": {
                            "$avg": {
                                "$multiply": [
                                    "$metrics.position",
                                    "$metrics.impressions",
                                ],
                            },
                        },
                        "total_impressions": {"$sum": "$metrics.impressions"},
                        "dates": {"$addToSet": "$date"},
                    },
                },
                {
                    "$addFields": {
                        "avg_position": {
                            "$cond": [
                                {"$eq": ["$total_impressions", 0]},
                                0,
                                {"$divide": ["$avg_position", "$total_impressions"]},
                            ],
                        },
                        "ctr": {
                            "$cond": [
                                {"$eq": ["$total_impressions", 0]},
                                0,
                                {"$divide": ["$clicks", "$total_impressions"]},
                            ],
                        },
                    },
                },
                {"$sort": {"clicks": -1}},
            ]
            response["query_performance"] = list(
                db.db.analytics.aggregate(query_pipeline)
            )

        if show_countries and results:
            # Get country breakdown
            country_pipeline = [
                {"$match": match_stage},
                {
                    "$group": {
                        "_id": "$dimensions.country",
                        "clicks": {"$sum": "$metrics.clicks"},
                        "impressions": {"$sum": "$metrics.impressions"},
                        "avg_position": {
                            "$avg": {
                                "$multiply": [
                                    "$metrics.position",
                                    "$metrics.impressions",
                                ],
                            },
                        },
                        "total_impressions": {"$sum": "$metrics.impressions"},
                        "dates": {"$addToSet": "$date"},
                    },
                },
                {
                    "$addFields": {
                        "avg_position": {
                            "$cond": [
                                {"$eq": ["$total_impressions", 0]},
                                0,
                                {"$divide": ["$avg_position", "$total_impressions"]},
                            ],
                        },
                        "ctr": {
                            "$cond": [
                                {"$eq": ["$total_impressions", 0]},
                                0,
                                {"$divide": ["$clicks", "$total_impressions"]},
                            ],
                        },
                    },
                },
                {"$sort": {"clicks": -1}},
            ]
            response["country_performance"] = list(
                db.db.analytics.aggregate(country_pipeline)
            )

        if show_devices and results:
            # Get device breakdown
            device_pipeline = [
                {"$match": match_stage},
                {
                    "$group": {
                        "_id": "$dimensions.device",
                        "clicks": {"$sum": "$metrics.clicks"},
                        "impressions": {"$sum": "$metrics.impressions"},
                        "avg_position": {
                            "$avg": {
                                "$multiply": [
                                    "$metrics.position",
                                    "$metrics.impressions",
                                ],
                            },
                        },
                        "total_impressions": {"$sum": "$metrics.impressions"},
                        "dates": {"$addToSet": "$date"},
                    },
                },
                {
                    "$addFields": {
                        "avg_position": {
                            "$cond": [
                                {"$eq": ["$total_impressions", 0]},
                                0,
                                {"$divide": ["$avg_position", "$total_impressions"]},
                            ],
                        },
                        "ctr": {
                            "$cond": [
                                {"$eq": ["$total_impressions", 0]},
                                0,
                                {"$divide": ["$clicks", "$total_impressions"]},
                            ],
                        },
                    },
                },
                {"$sort": {"clicks": -1}},
            ]
            response["device_performance"] = list(
                db.db.analytics.aggregate(device_pipeline)
            )

        return response

    except Exception as e:
        raise HTTPException(status_code=500, detail=str(e))


In [ ]:
#| export
from pydantic import BaseModel, Field, constr
from typing import Optional, List, Dict, Any
from enum import Enum


class CountryCode(str, Enum):
    """Country codes from Google Search Console

    Three-letter ISO 3166-1 alpha-3 codes used in Search Console data.
    Common regions are also included (like EU for European Union).

    Reference: https://en.wikipedia.org/wiki/ISO_3166-1_alpha-3
    """

    # Middle East
    SAU = "sau"  # Saudi Arabia
    ARE = "are"  # United Arab Emirates
    EGY = "egy"  # Egypt
    QAT = "qat"  # Qatar
    BHR = "bhr"  # Bahrain
    KWT = "kwt"  # Kuwait
    OMN = "omn"  # Oman
    YEM = "yem"  # Yemen
    IRQ = "irq"  # Iraq
    JOR = "jor"  # Jordan
    LBN = "lbn"  # Lebanon
    SYR = "syr"  # Syria

    # North Africa
    MAR = "mar"  # Morocco
    DZA = "dza"  # Algeria
    TUN = "tun"  # Tunisia
    LBY = "lby"  # Libya
    SDN = "sdn"  # Sudan

    # Europe
    GBR = "gbr"  # United Kingdom
    DEU = "deu"  # Germany
    FRA = "fra"  # France
    ITA = "ita"  # Italy
    ESP = "esp"  # Spain
    NLD = "nld"  # Netherlands
    POL = "pol"  # Poland
    RUS = "rus"  # Russia

    # Americas
    USA = "usa"  # United States
    CAN = "can"  # Canada
    MEX = "mex"  # Mexico
    BRA = "bra"  # Brazil
    ARG = "arg"  # Argentina

    # Asia
    CHN = "chn"  # China
    JPN = "jpn"  # Japan
    KOR = "kor"  # South Korea
    IND = "ind"  # India
    IDN = "idn"  # Indonesia
    MYS = "mys"  # Malaysia
    SGP = "sgp"  # Singapore
    THA = "tha"  # Thailand
    VNM = "vnm"  # Vietnam
    PHL = "phl"  # Philippines

    # Africa
    ZAF = "zaf"  # South Africa
    NGA = "nga"  # Nigeria
    KEN = "ken"  # Kenya
    GHA = "gha"  # Ghana

    # Oceania
    AUS = "aus"  # Australia
    NZL = "nzl"  # New Zealand

    # Other Regions
    EUR = "eur"  # European Union
    ASI = "asi"  # Asia
    AFR = "afr"  # Africa
    NAM = "nam"  # North America
    SAM = "sam"  # South America
    OCE = "oce"  # Oceania

    @classmethod
    def get_name(cls, code: str) -> str:
        """Get country name from code

        Args:
            code: Three-letter country code

        Returns:
            Full country name or original code if not found
        """
        names = {
            "sau": "Saudi Arabia",
            "are": "United Arab Emirates",
            "egy": "Egypt",
            "qat": "Qatar",
            "bhr": "Bahrain",
            "kwt": "Kuwait",
            "omn": "Oman",
            "yem": "Yemen",
            "irq": "Iraq",
            "jor": "Jordan",
            "lbn": "Lebanon",
            "syr": "Syria",
            "mar": "Morocco",
            "dza": "Algeria",
            "tun": "Tunisia",
            "lby": "Libya",
            "sdn": "Sudan",
            "gbr": "United Kingdom",
            "deu": "Germany",
            "fra": "France",
            "ita": "Italy",
            "esp": "Spain",
            "nld": "Netherlands",
            "pol": "Poland",
            "rus": "Russia",
            "usa": "United States",
            "can": "Canada",
            "mex": "Mexico",
            "bra": "Brazil",
            "arg": "Argentina",
            "chn": "China",
            "jpn": "Japan",
            "kor": "South Korea",
            "ind": "India",
            "idn": "Indonesia",
            "mys": "Malaysia",
            "sgp": "Singapore",
            "tha": "Thailand",
            "vnm": "Vietnam",
            "phl": "Philippines",
            "zaf": "South Africa",
            "nga": "Nigeria",
            "ken": "Kenya",
            "gha": "Ghana",
            "aus": "Australia",
            "nzl": "New Zealand",
            "eur": "European Union",
            "asi": "Asia",
            "afr": "Africa",
            "nam": "North America",
            "sam": "South America",
            "oce": "Oceania",
        }
        return names.get(code.lower(), code)


class CountryMetrics(MetricsData):
    """Country-specific metrics with dimensions"""

    country: str = Field(..., description="Country code")
    dates: List[str] = Field(default_factory=list, description="Dates with data")
    queries: List[str] = Field(default_factory=list, description="Queries from country")
    pages: List[str] = Field(
        default_factory=list, description="Pages viewed from country"
    )
    devices: List[DeviceType] = Field(default_factory=list, description="Devices used")


class CountryAnalyticsResponse(BaseModel):
    """Response model for country analytics

    Attributes:
        site_url: Website being analyzed
        country: Country being analyzed (or 'all')
        date_range: Information about the analysis period
        countries_performance: Performance data by country
        top_queries: Optional top queries from country
        top_pages: Optional top pages in country
        device_breakdown: Optional device usage breakdown
    """

    site_url: str
    country: str
    date_range: Dict[str, Any]
    countries_performance: List[CountryMetrics]
    top_queries: Optional[List[MetricsData]]
    top_pages: Optional[List[MetricsData]]
    device_breakdown: Optional[List[MetricsData]]


@app.get(
    "/country-analytics/{site_url}",
    response_model=CountryAnalyticsResponse,
    summary="Analyze country performance",
    description="Analyze performance by country with detailed breakdowns",
    tags=["analytics"],
)
async def country_analytics(
    site_url: str,  # Website URL to analyze
    country: Optional[str] = Query(
        default=None,
        description="Country code to analyze (e.g., 'sau', 'egy')",
        min_length=3,
        max_length=3,
        regex="^[a-zA-Z]{3}$",
    ),
    range_type: str = Query(
        default="today",
        description="Date range type: today, last_days, last_months, custom",
    ),
    days: Optional[int] = Query(
        default=None, ge=1, le=365, description="Number of days for last_days range"
    ),
    months: Optional[int] = Query(
        default=None, ge=1, le=12, description="Number of months for last_months range"
    ),
    start_date: Optional[str] = Query(
        default=None, description="Custom start date (YYYY-MM-DD)"
    ),
    end_date: Optional[str] = Query(
        default=None, description="Custom end date (YYYY-MM-DD)"
    ),
    show_queries: bool = Query(
        default=True, description="Include top queries from country"
    ),
    show_pages: bool = Query(default=True, description="Include top pages in country"),
    show_devices: bool = Query(default=False, description="Include device breakdown"),
) -> CountryAnalyticsResponse:
    """Analyze performance by country over time.

    Retrieves detailed performance metrics for countries, with optional
    focus on a specific country. Includes breakdowns by queries, pages,
    and devices when requested.

    Args:
        site_url: Website to analyze
        country: Optional 3-letter country code (e.g., 'sau', 'egy')
        range_type: Type of date range (today, last_days, last_months, custom)
        days: Number of days for last_days range
        months: Number of months for last_months range
        start_date: Start date for custom range (YYYY-MM-DD)
        end_date: End date for custom range (YYYY-MM-DD)
        show_queries: Include top queries breakdown
        show_pages: Include top pages breakdown
        show_devices: Include device breakdown

    Returns:
        CountryAnalyticsResponse containing:
        - Overall country performance
        - Optional top queries
        - Optional top pages
        - Optional device breakdown

    Raises:
        HTTPException:
            - 400 for invalid parameters
            - 500 for database errors

    Examples:
        >>> # Get all countries performance
        >>> response = await country_analytics(
        ...     "sc-domain:example.com",
        ...     range_type="last_days",
        ...     days=7
        ... )

        >>> # Analyze specific country with all breakdowns
        >>> response = await country_analytics(
        ...     "sc-domain:example.com",
        ...     country="sau",
        ...     show_queries=True,
        ...     show_pages=True,
        ...     show_devices=True
        ... )

    Notes:
        - Country codes are case-insensitive
        - Position metrics are weighted by impressions
        - CTR is calculated as clicks/impressions
        - Top queries/pages limited to 100 results
    """
    try:
        # Get date range using utility function
        start_date, end_date = get_date_range(
            range_type=range_type,
            days=days,
            months=months,
            start_date=start_date,
            end_date=end_date,
        )

        # Base match condition
        match_stage = {
            "site_url": site_url,
            "date": {
                "$gte": start_date,
                "$lte": end_date,
            },
        }

        # Add country filter if specified
        if country:
            match_stage["dimensions.country"] = {
                "$regex": f"^{country}$",
                "$options": "i",
            }

        # Main country pipeline
        pipeline = [
            {"$match": match_stage},
            {
                "$group": {
                    "_id": "$dimensions.country",
                    "clicks": {"$sum": "$metrics.clicks"},
                    "impressions": {"$sum": "$metrics.impressions"},
                    "avg_position": {
                        "$avg": {
                            "$multiply": ["$metrics.position", "$metrics.impressions"],
                        },
                    },
                    "total_impressions": {"$sum": "$metrics.impressions"},
                    "dates": {"$addToSet": "$date"},
                    "queries": {"$addToSet": "$dimensions.query"},
                    "pages": {"$addToSet": "$dimensions.page"},
                    "devices": {"$addToSet": "$dimensions.device"},
                },
            },
            {
                "$addFields": {
                    "avg_position": {
                        "$cond": [
                            {"$eq": ["$total_impressions", 0]},
                            0,
                            {"$divide": ["$avg_position", "$total_impressions"]},
                        ],
                    },
                    "ctr": {
                        "$cond": [
                            {"$eq": ["$total_impressions", 0]},
                            0,
                            {"$divide": ["$clicks", "$total_impressions"]},
                        ],
                    },
                    "date_count": {"$size": "$dates"},
                },
            },
            {"$sort": {"clicks": -1}},
        ]

        results = list(db.db.analytics.aggregate(pipeline))

        response = {
            "site_url": site_url,
            "country": country if country else "all",
            "date_range": {
                "start": start_date,
                "end": end_date,
                "type": range_type,
                "total_days": len(set(date for r in results for date in r["dates"])),
            },
            "countries_performance": results,
        }

        # If specific country is selected, add detailed breakdowns
        if country and results:
            if show_queries:
                # Get top queries from this country
                query_pipeline = [
                    {"$match": match_stage},
                    {
                        "$group": {
                            "_id": "$dimensions.query",
                            "clicks": {"$sum": "$metrics.clicks"},
                            "impressions": {"$sum": "$metrics.impressions"},
                            "avg_position": {
                                "$avg": {
                                    "$multiply": [
                                        "$metrics.position",
                                        "$metrics.impressions",
                                    ],
                                },
                            },
                            "total_impressions": {"$sum": "$metrics.impressions"},
                        },
                    },
                    {
                        "$addFields": {
                            "avg_position": {
                                "$cond": [
                                    {"$eq": ["$total_impressions", 0]},
                                    0,
                                    {
                                        "$divide": [
                                            "$avg_position",
                                            "$total_impressions",
                                        ]
                                    },
                                ],
                            },
                            "ctr": {
                                "$cond": [
                                    {"$eq": ["$total_impressions", 0]},
                                    0,
                                    {"$divide": ["$clicks", "$total_impressions"]},
                                ],
                            },
                        },
                    },
                    {"$sort": {"clicks": -1}},
                    {"$limit": 100},
                ]
                response["top_queries"] = list(
                    db.db.analytics.aggregate(query_pipeline)
                )

            if show_pages:
                # Get top pages in this country
                page_pipeline = [
                    {"$match": match_stage},
                    {
                        "$group": {
                            "_id": "$dimensions.page",
                            "clicks": {"$sum": "$metrics.clicks"},
                            "impressions": {"$sum": "$metrics.impressions"},
                            "avg_position": {
                                "$avg": {
                                    "$multiply": [
                                        "$metrics.position",
                                        "$metrics.impressions",
                                    ],
                                },
                            },
                            "total_impressions": {"$sum": "$metrics.impressions"},
                        },
                    },
                    {
                        "$addFields": {
                            "avg_position": {
                                "$cond": [
                                    {"$eq": ["$total_impressions", 0]},
                                    0,
                                    {
                                        "$divide": [
                                            "$avg_position",
                                            "$total_impressions",
                                        ]
                                    },
                                ],
                            },
                            "ctr": {
                                "$cond": [
                                    {"$eq": ["$total_impressions", 0]},
                                    0,
                                    {"$divide": ["$clicks", "$total_impressions"]},
                                ],
                            },
                        },
                    },
                    {"$sort": {"clicks": -1}},
                    {"$limit": 100},
                ]
                response["top_pages"] = list(db.db.analytics.aggregate(page_pipeline))

            if show_devices:
                # Get device breakdown for this country
                device_pipeline = [
                    {"$match": match_stage},
                    {
                        "$group": {
                            "_id": "$dimensions.device",
                            "clicks": {"$sum": "$metrics.clicks"},
                            "impressions": {"$sum": "$metrics.impressions"},
                            "avg_position": {
                                "$avg": {
                                    "$multiply": [
                                        "$metrics.position",
                                        "$metrics.impressions",
                                    ],
                                },
                            },
                            "total_impressions": {"$sum": "$metrics.impressions"},
                        },
                    },
                    {
                        "$addFields": {
                            "avg_position": {
                                "$cond": [
                                    {"$eq": ["$total_impressions", 0]},
                                    0,
                                    {
                                        "$divide": [
                                            "$avg_position",
                                            "$total_impressions",
                                        ]
                                    },
                                ],
                            },
                            "ctr": {
                                "$cond": [
                                    {"$eq": ["$total_impressions", 0]},
                                    0,
                                    {"$divide": ["$clicks", "$total_impressions"]},
                                ],
                            },
                        },
                    },
                    {"$sort": {"clicks": -1}},
                ]
                response["device_breakdown"] = list(
                    db.db.analytics.aggregate(device_pipeline)
                )

        return response

    except Exception as e:
        raise HTTPException(status_code=500, detail=str(e))
